## The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
import glob

# The workers

In [2]:
ask_workers=20
memory='180GB'

In [3]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = PBSCluster(cores=1, memory=memory, project='PerfTestPangeo', walltime='00:30:00')
cluster.scale(ask_workers)

c = Client(cluster)

c


/home/ad/alberta/.conda/envs/perf-pangeo/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33094 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://10.120.43.21:36332 Dashboard: http://10.120.43.21:33094/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 20, Memory=3.60 TB


# The data

In [5]:
%time ds=xr.open_zarr('/work/ALT/odatis/eNATL60/zarr/eNATL60-BLBT02-SSH-1h-mballaro')

CPU times: user 114 ms, sys: 34.7 ms, total: 149 ms
Wall time: 158 ms


In [6]:
ds

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, time_counter: 11688, x: 8354, y: 4729)
Coordinates:
    nav_lat               (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    time_centered         (time_counter) datetime64[ns] dask.array<chunksize=(11688,), meta=np.ndarray>
  * time_counter          (time_counter) datetime64[ns] 2009-06-30T00:30:00 ... 2010-10-29T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    sossheig              (time_counter, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(11688, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(11688, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    TimeStamp:    08/01/2019 09:34:21 +0100
    description:  ocean T grid variables
    file_name:    eNATL60-BLBT02_1h_20090630_20090704_gridT-2D_20090630-20090...
    ibegin:       0
    jbegin:       0
    name:         /scratch/tmp/3746956/eNATL60-BLBT02_1h_20090630_20090704_gr...
    ni:           8354
    nj:           9
    timeStamp:    2018-Dec-31 17:48:48 GMT
    title:        ocean T grid variables
    uuid:         bd4ccb1a-e144-450d-b3d7-330382974e7a

In [7]:
%time tmean=ds.sossheig.mean(dim='time_counter')

CPU times: user 149 ms, sys: 6.84 ms, total: 156 ms
Wall time: 163 ms


In [8]:
tmean

<xarray.DataArray 'sossheig' (y: 4729, x: 8354)>
dask.array<mean_agg-aggregate, shape=(4729, 8354), dtype=float32, chunksize=(4729, 8354), chunktype=numpy.ndarray>
Coordinates:
    nav_lat  (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    nav_lon  (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
Dimensions without coordinates: y, x

In [9]:
%time tmean.load()

CPU times: user 3min 36s, sys: 8.4 s, total: 3min 45s
Wall time: 14min 39s


<xarray.DataArray 'sossheig' (y: 4729, x: 8354)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
    nav_lat  (y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    nav_lon  (y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
Dimensions without coordinates: y, x

In [6]:
%time smean=ds.sossheig.mean(dim={'x','y'})

CPU times: user 286 ms, sys: 6.81 ms, total: 293 ms
Wall time: 300 ms


In [7]:
smean

,Array,Chunk
Bytes,46.75 kB,4 B
Shape,"(11688,)","(1,)"
Count,35065 Tasks,11688 Chunks
Type,float32,numpy.ndarray


In [8]:
%time smean.load()

CPU times: user 1min 35s, sys: 4.55 s, total: 1min 39s
Wall time: 4min 53s


<xarray.DataArray 'sossheig' (time_counter: 11688)>
array([-0.2624576 , -0.26359338, -0.2584397 , ..., -0.22629748,
       -0.24492481, -0.25286493], dtype=float32)
Coordinates:
    time_centered  (time_counter) datetime64[ns] 2009-06-30T00:30:00 ... 2010-10-29T23:30:00
  * time_counter   (time_counter) datetime64[ns] 2009-06-30T00:30:00 ... 2010-10-29T23:30:00

In [9]:
cluster.close()